# LAB 6:  Serving baby weight predictions

**Learning Objectives**

1. Deploy a web application that consumes your model service on Cloud AI Platform.

## Introduction 
**Verify that you have previously Trained your Keras model and Deployed it predicting with Keras model on Cloud AI Platform. If not, go back to [5a_train_keras_ai_platform_babyweight.ipynb](../solutions/5a_train_keras_ai_platform_babyweight.ipynb) and [5b_deploy_keras_ai_platform_babyweight.ipynb](../solutions/5b_deploy_keras_ai_platform_babyweight.ipynb) create them.**

In the previous notebook, we deployed our model to CAIP. In this notebook, we'll make a [Flask app](https://palletsprojects.com/p/flask/) to show how our models can interact with a web application which could be deployed to [App Engine](https://cloud.google.com/appengine) with the [Flexible Environment](https://cloud.google.com/appengine/docs/flexible).

## Step 1: Review Flask App code in `application` folder
Let's start with what our users will see. In the `application` folder, we have prebuilt the components for web application. In the templates folder, the <a href="application/templates/index.html">index.html</a> file is the visual GUI our users will make predictions with.

It works by using an HTML [form](https://www.w3schools.com/html/html_forms.asp) to make a [POST request](https://www.w3schools.com/tags/ref_httpmethods.asp) to our server, passing along the values captured by the [input tags](https://www.w3schools.com/html/html_form_input_types.asp).

The form will render a little strangely in the notebook since the notebook environment does not run javascript, nor do we have our web server up and running. Let's get to that!

## Step 2: Set environment variables

In [16]:
import os
out = os.popen('gcloud config list project --format "value(core.project)"')
PROJECT = out.read().strip()
BUCKET = PROJECT

In [17]:
os.environ["PROJECT"] = PROJECT
os.environ["BUCKET"] = BUCKET

## Step 3: Complete application code in `application/main.py`
We can set up our server with python using [Flask](https://flask.palletsprojects.com/en/1.1.x/quickstart/). Below, we've already built out most of the application for you.

The `@app.route()` decorator defines a function to handle web reqests. Let's say our website is `www.example.com`. With how our `@app.route("/")` function is defined, our sever will render our <a href="application/templates/index.html">index.html</a> file when users go to `www.example.com/` (which is the default route for a website).

So, when a user pings our server with `www.example.com/predict`, they would use `@app.route("/predict", methods=["POST"])` to make a prediction. The data that gets sent over the internet isn't a dictionary, but a string like below:

`name1=value1&name2=value2` where `name` corresponds to the `name` on the input tag of our html form, and the value is what the user entered. Thankfully, Flask makes it easy to transform this string into a dictionary with `request.form.to_dict()`, but we still need to transform the data into a format our model expects. We've done this with the `gender2str` and the `plurality2str` utility functions.

Ok! Let's set up a webserver to take in the form inputs, process them into features, and send these features to our model on Cloud AI Platform to generate predictions to serve to back to users.

Fill in the **TODO** comments in <a href="application/main.py">application/main.py</a>. Give it a go first and review the solutions folder if you get stuck.

**Note:** AppEngine test configurations have already been set for you in the file <a href="application/app.yaml">application/app.yaml</a>. Review [app.yaml](https://cloud.google.com/appengine/docs/standard/python/config/appref) documentation for additional configuration options.

## Step 4: Deploy application
So how do we know that it works? We'll have to deploy our website and find out! Notebooks aren't made for website deployment, so we'll move our operation to the [Google Cloud Shell](https://console.cloud.google.com/home/dashboard?cloudshell=true).

By default, the shell doesn't have Flask installed, so copy over the following command to install it.

`python3 -m pip install --user Flask==0.12.1`

Next, we'll need to copy our web app to the Cloud Shell. We can use [Google Cloud Storage](https://cloud.google.com/storage) as an inbetween.

In [18]:
!gsutil mb gs://$BUCKET

Creating gs://qwiklabs-gcp-00-0db9b1bc58c6/...
ServiceException: 409 A Cloud Storage bucket named 'qwiklabs-gcp-00-0db9b1bc58c6' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.


In [19]:
!echo "gs://$BUCKET/asl-capstone"

gs://qwiklabs-gcp-00-0db9b1bc58c6/asl-capstone


In [20]:
%%bash
gsutil -m rm -r gs://$BUCKET/asl-capstone
gsutil -m cp -r ./ gs://$BUCKET/asl-capstone

CommandException: 1 files/objects could not be removed.
Copying file://./main.py [Content-Type=text/x-python]...
Copying file://./deploy_web.ipynb [Content-Type=application/octet-stream]...
Copying file://./car_spec.csv [Content-Type=text/csv]...
Copying file://./refined_used_car.csv [Content-Type=text/csv]...
Copying file://./templates/index.html [Content-Type=text/html]...
Copying file://./requirements.txt [Content-Type=text/plain]...
Copying file://./handle_csv.py [Content-Type=text/x-python]...
Copying file://./.ipynb_checkpoints/deploy_web-checkpoint.ipynb [Content-Type=application/octet-stream]...
Copying file://./.vscode/settings.json [Content-Type=application/json]...
Copying file://./templates/result.html [Content-Type=text/html]...
Copying file://./__pycache__/config.cpython-37.pyc [Content-Type=application/x-python-code]...
Copying file://./.git/index [Content-Type=application/octet-stream]...
Copying file://./app.yaml [Content-Type=application/octet-stream]...
Copying file:

Run the below cell, and copy the output into the [Google Cloud Shell](https://console.cloud.google.com/home/dashboard?cloudshell=true)

In [21]:
%%bash
pip3 install pandas
echo rm -r asl-capstone/
echo mkdir asl-capstone/
echo gsutil cp -r gs://$BUCKET/asl-capstone ./
echo python3 asl-capstone/main.py

rm -r asl-capstone/
mkdir asl-capstone/
gsutil cp -r gs://qwiklabs-gcp-00-0db9b1bc58c6/asl-capstone ./
python3 asl-capstone/main.py


## Step 5: Use your website to generate predictions
Time to play with the website! The cloud shell should now say something like `* Running on http://127.0.0.1:8080/ (Press CTRL+C to quit)`. Click on the `http` link to go to your shiny new website. Fill out the form and give it a minute or two to process its first prediction. After the first one, the rest of the predictions will be lightning fast.

Did you get a prediction? If not, the Google Cloud Shell will spit out a stack trace of the error to help narrow it down. If yes, congratulations! Great job on bringing all of your work together for the users.

## Lab Summary



In this lab, you deployed a simple Flask web form application on App Engine that takes inputs, transforms them into features, and sends them to a model service on Cloud AI Platform to generate and return predictions.

Copyright 2019 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License